In [2]:
import pandas as pd
import numpy as np
import os
import pickle
from tqdm import tqdm

import pyaudio  
import wave  

import shutil

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from collections import Counter

import json

import soundfile as sf

In [64]:
path = 'data\\train_opus\\'

In [65]:
tweets = []
name_json = ['1hour.jsonl', '10hour.jsonl', '100hour.jsonl', '10min.jsonl', 'manifest.jsonl']

for line in open(path + name_json[-1], 'r'):
    tweets.append(json.loads(line))

In [66]:
tweets[0]

{'id': '77b380796d242cf5bc09cfb551cffecd',
 'audio_filepath': 'crowd/0/77b380796d242cf5bc09cfb551cffecd.opus',
 'text': 'алиби',
 'duration': 1.21}

# Ф-я преобразования цифр в строчные значения

In [3]:
def as_words(n):
    """Convert an integer n (+ve or -ve) to English words."""
    # lookups
    ones = ['ноль', 'один', 'два', 'три', 'четыре',
            'пять', 'шесть', 'семь', 'восемь', 'девять', 
            'десять', 'одиннадцать', 'двенадцать', 'тринадцать', 'четырнадцать',
            'пятнадцать', 'шестнадцать', 'семнадцать', 'восемнадцать', 'девятнадцать']
    tens = ['ноль', 'десять', 'двадцать', 'тридцать', 'сорок',
            'пятьдесят', 'шестьдесяст', 'семьдесят', 'восемьдесят', 'девяносто']
    # negative case
    if n < 0:
        return "минус {0}".format(as_words(abs(n)))
    # 1000+
    for order, word in [(10**12, "трилион"), (10**9, "биллион"),
                        (10**6, "миллион"), (10**3, "тысячи")]:
        if n >= order:
            return "{0} {1}{2}".format(as_words(n // order), word,
                                       " {0}".format(as_words(n % order))
                                       if n % order else "")
    # 100-999
    if n >= 100:
        if n / 300 > 1:
            return "триста {1}".format(as_words(n // 100), 
                                                as_words(n % 100))
        elif n / 300 == 1:
            return "триста"
        elif n / 200 >= 1:
            return "двести".format(as_words(n // 100), 
                                                as_words(n % 100))
        elif n / 200 == 1:
            return "двести"
        elif n / 100 > 1:
            return "сто {1}".format(as_words(n // 100), 
                                                as_words(n % 100))
        elif n / 100 == 1:
            return "сто"
        
        else:
            return "{0} сто".format(as_words(n // 100))
    # 0-99
    if n < 20:
        return ones[n]
    else:
        return "{0}{1}".format(tens[n // 10],
                               " {0}".format(as_words(n % 10)) 
                               if n % 10 else "") 

In [4]:
main_word = []

In [5]:
word = ['килина', 'ванна','кухня','туалет','коридор','детская','спальня', 'включить', 'выключить']
# word = ['включить', 'выключить', 'точка', 'привет', "мотодепо", 'пока', 'вверх', 'вниз', 'влево', "вправо"]

In [6]:
number = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30,40,50,60,70,80,90,
         100,200, 300]

In [7]:
for i in number:
    num = as_words(i)
#     print(num, i)
#     print('*' * 20)
    main_word.append(num)
len(main_word)

31

In [8]:
for i in word:
    main_word.append(i)
len(main_word)    

40

## Поиск слов в датасете

In [9]:
np.array(main_word)

array(['ноль', 'один', 'два', 'три', 'четыре', 'пять', 'шесть', 'семь',
       'восемь', 'девять', 'десять', 'одиннадцать', 'двенадцать',
       'тринадцать', 'четырнадцать', 'пятнадцать', 'шестнадцать',
       'семнадцать', 'восемнадцать', 'девятнадцать', 'двадцать',
       'тридцать', 'сорок', 'пятьдесят', 'шестьдесяст', 'семьдесят',
       'восемьдесят', 'девяносто', 'сто', 'двести', 'триста', 'килина',
       'ванна', 'кухня', 'туалет', 'коридор', 'детская', 'спальня',
       'включить', 'выключить'], dtype='<U12')

In [10]:
main_word_norm = [morph.parse(i)[0] for i in main_word]
main_word_norm = [i.normal_form for i in main_word_norm]

# word_norm = morph.parse(word)[0]
# word_norm = word_norm.normal_form

In [11]:
np.array(main_word_norm)

array(['ноль', 'один', 'два', 'три', 'четыре', 'пять', 'шесть', 'семь',
       'восемь', 'девять', 'десять', 'одиннадцать', 'двенадцать',
       'тринадцать', 'четырнадцать', 'пятнадцать', 'шестнадцать',
       'семнадцать', 'восемнадцать', 'девятнадцать', 'двадцать',
       'тридцать', 'сорок', 'пятьдесят', 'шестьдесястый', 'семьдесят',
       'восемьдесят', 'девяносто', 'сто', 'двести', 'триста', 'килина',
       'ванна', 'кухня', 'туалет', 'коридор', 'детский', 'спальня',
       'включить', 'выключить'], dtype='<U13')

In [11]:
# 'manifest.jsonl'
tweets = []
res = {}
count = 1
for dict_ in tweets:
    text = dict_['text']
    if len(text.split()) == 1:
        word_norm = morph.parse(text)[0]
        word_norm = word_norm.normal_form
        if text in main_word_norm:
            print(f"{name} : {text}")
            if text in res:
                res[text] += 1
            else:
                res[text] = 1
print('manifest.jsonl')       
print(res)

manifest.jsonl
{}


In [27]:
with open("data\\train_opus\\crowd\\test.txt", "w") as file:
    file.write("tes test test")

In [29]:
path = 'data\\train_opus\\'

name_json = ['1hour.jsonl', '10hours.jsonl', '100hours.jsonl', '10min.jsonl', 'manifest.jsonl']
count_one = 0
count_all = 0
for name in name_json:
    print(name)
    tweets = []
    for line in open(path + name, 'r'):
        tweets.append(json.loads(line))
        res = {}
    for dict_ in tweets:
        text = dict_['text']
        path_file = dict_['audio_filepath']
        path_file = path + path_file
        name = dict_['id']
        if len(text.strip().split()) == 1:
    #         print(f"{name} : {text} len == 1")
            word_norm = morph.parse(text)[0]
            word_norm = word_norm.normal_form
            if word_norm in main_word_norm:
    #                     print(f"{name} : {text}")
                count_all += 1
                if text in res:
                    res[text] += 1
                else:
                    res[text] = 1
                shutil.copy(path_file, 'result_data\\2')
                with open(f"result_data\\2\\{name}.txt", "w") as file:
                    file.write(f"{text}")
#                 shutil.copy(path_txt, 'result_all_data')
#     print(name)       
    print(res)
count_all

1hour.jsonl
{}
10hours.jsonl
{'выключи': 9, 'один': 1, 'восемьдесят': 1, 'детская': 1}
100hours.jsonl
{'выключи': 96, 'два': 1, 'включи': 1, 'один': 1, 'девяносто': 1, 'детский': 3, 'пятнадцать': 1, 'кухня': 1, 'пять': 1, 'детские': 1}
10min.jsonl
{}
manifest.jsonl
{'семь': 3, 'один': 6, 'кухня': 14, 'пятнадцать': 2, 'включи': 13, 'четыре': 4, 'три': 5, 'два': 3, 'пять': 3, 'детский': 68, 'детские': 3, 'семнадцать': 1, 'одна': 1, 'шесть': 1, 'девять': 1, 'одни': 1, 'двадцать': 1, 'включу': 1, 'тринадцать': 2, 'тридцать': 1, 'одиннадцать': 1, 'девятнадцать': 2, 'восемьдесят': 1, 'выключи': 933, 'детская': 12, 'килине': 1, 'девяносто': 1}


1204

In [30]:
os.path.exists("crowd/8/86b9b825fe0f2a6bba4f2765bdba1bc7.wav")

False

In [50]:
path = 'data\\'

name_json = ['1hour.jsonl', '10hours.jsonl', '100hours.jsonl', '10min.jsonl', 'manifest.jsonl']
count_one = 0
count_all = 0
for name in name_json:
    print(name)
    tweets = []
    
    for line in open(path + name, 'r', encoding = 'utf-8'):
        tweets.append(json.loads(line))
        res = {}
#         print(line)
    for dict_ in tweets:
        one_twe = []
        text = dict_['text']
        audio_filepath = dict_['audio_filepath']
        audio_filepath = 'data//' + audio_filepath
        name = dict_['id']
#         print(audio_filepath, audio_filepath.split('/')[1])
        if audio_filepath.split('/')[1] == 0 and len(text.split()) == 1:
            print(text)
        if len(text.strip().split()) == 1:
            if os.path.exists(audio_filepath):
#                 print(1)
                word_norm = morph.parse(text)[0]
                word_norm = word_norm.normal_form
                if word_norm in main_word_norm:
#                     print(audio_filepath)
                    if text in res:
                        res[text] += 1
                    else:
                        res[text] = 1
                
                    shutil.copy(audio_filepath, 'result_data\\2')
                    with open(f"result_data\\2\\{name}.txt", "w") as file:
                        file.write(f"{word_norm}")
            
    print(res)

1hour.jsonl
{}
10hours.jsonl
{'выключи': 9, 'детская': 1}
100hours.jsonl
{'выключи': 96, 'два': 1, 'включи': 1}
10min.jsonl
{}
manifest.jsonl
{'четыре': 1, 'три': 3, 'семь': 1, 'два': 1, 'пять': 1, 'один': 1, 'выключи': 933, 'детская': 12, 'включи': 3, 'килине': 1}


In [34]:
path = 'data\\train_opus\\'

name_json = ['1hour.jsonl', '10hours.jsonl', '100hours.jsonl', '10min.jsonl', 'manifest.jsonl']
count_one = 0
count_all = 0
for name in name_json:
    print(name)
    tweets = []
    for line in open(path + name, 'r'):
        tweets.append(json.loads(line))
        res = {}
    for dict_ in tweets:
        text = dict_['text']
        path = dict_['audio_filepath']
        if len(text.strip().split()) == 1:
    #         print(f"{name} : {text} len == 1")
            word_norm = morph.parse(text)[0]
            word_norm = word_norm.normal_form
            if word_norm in main_word_norm:
    #                     print(f"{name} : {text}")
                count_all += 1
                if text in res:
                    res[text] += 1
                else:
                    res[text] = 1
#     print(name)       
    print(res)
count_all

1hour.jsonl


FileNotFoundError: [Errno 2] No such file or directory: 'data\\train_opus\\1hour.jsonl'

In [26]:
name_json = ['1hour.jsonl', '10hours.jsonl', '100hours.jsonl', '10min.jsonl', 'manifest.jsonl']
tweets = []
res = {}
count = 1

for line in open(path + name_json[0], 'r'):
    tweets.append(json.loads(line))
print(tweets[0])

{'id': 'cadbe57fe680d0581348a172820af423', 'audio_filepath': 'crowd/8/cadbe57fe680d0581348a172820af423.opus', 'text': 'улица молодежная дом семнадцать в', 'duration': 4.3649375}


# opus to wav

In [ ]:
def convert_opus_to_wav(data):
    for index in data.index:  # Пробегаем по встроенному манифесту датасета
        file = r"data\\train_opus\\crowd\\" + data.loc[index, "Файлы"]  # Запоминаем путь к opus файлу
        if os.path.exists(file):  # Если файл есть, то преобразовываем
            audio, sample_rate = sf.read(file, dtype='int16')  # Читаем opus
            sf.write(file.replace(".opus", ".wav"), audio, sample_rate)  # Сохраняем wav
            os.remove(file)  # Заметаем следы (Удаляем преобразованный файл)

In [37]:
file = r'result_data\\2\\0011d821437ae7e0afae1ac7cc260adb.opus'
audio, sample_rate = sf.read(file, dtype='int16')  # Читаем opus
sf.write(file.replace(".opus", ".wav"), audio, sample_rate)  # Сохраняем wav
os.remove(file)  # Заметаем следы (Удаляем преобразованный файл)


LibsndfileError: Supported file format but file is malformed.

In [43]:
path = r"C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2"
for root, dirs, files in os.walk(path):
    for file in files:
        if file.split('.')[-1] == 'opus':
            file = path + "\\" + file
            
            try:
                audio, sample_rate = sf.read(file, dtype='int16')  # Читаем opus
                sf.write(file.replace(".opus", ".wav"), audio, sample_rate)  # Сохраняем wav
                os.remove(file)  # Заметаем следы (Удаляем преобразованный файл)
            except:
                print(file)
                continue
#     print(files)
    

C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\0011d821437ae7e0afae1ac7cc260adb.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\0044415a449effd7ff7f74f7b84e9208.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\0086ab3047eb2daf577558d16b02e293.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\00d9154290151f4e421a734ff44a7e1a.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\01075307321413c5ed304b417f1d54f0.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\01348e5ec392fe3d50f15c33155fa064.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\0172a30da09de0b57510cebbd1492dfe.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\01b3068806ae73ea4badaa66ba8d78a6.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\023a33414ed07ef28dc7b685204ea3bb.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\02953c6c5a89929d3122a5227ca1c3e2.opus
C:\Users\VoronkovSer

C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\1cb6b1e956e2c5a142e3e0757e70a1b3.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\1cd61cb35b0a476f0ceaf393d1214f2b.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\1d109e6ef1f22ea6fca7fde4c553e684.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\1d323e85b788de6f2084470657b75af7.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\1def19cfcddcce5210b7fb20389fee58.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\1e17865708ad57efa039fb9e73cdb667.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\1ee52a2495ffa5dc90dde00b3b78d389.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\1fc037adde19c9410c52df94d729f497.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\1ffa577e658925b108cffa4a6b16fdfb.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\205dcae0ba793e869f12f36713e95c42.opus
C:\Users\VoronkovSer

C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\3b511106a5b484f5cb80b6bbf47ffaf8.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\3b77a693e246fd54fbd1021fb3be8edb.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\3bb6eee4e138ceb8ed610efc315fee39.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\3bd8dfa4cd3121d076c67bfe9e48f0f8.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\3bfe7e8e8dd3f787f37919ff140f5f5e.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\3ccd12aade30459a0f00eb988edb630b.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\3cf3d001404bdad09edcb27dc2aec9f2.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\3d72b94413bf4e5e153a0e748b098f6a.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\3ddd707f8cccb7e2cad080cbc39b426f.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\3e82d33bbe98ea5787f28369b4f3b7d0.opus
C:\Users\VoronkovSer

C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\54904c2ba64b78cbfd113247e4e529c8.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\554ab9d14897d5424136732b69abb59a.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\557712b629d0a73834ce16d864781818.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\557d4021dabb689a7c75675b735a2895.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\5682c6b1284d08b5b5de799512314c5e.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\56bc1a3d6bb330811ced053ae49b32a5.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\5721bbceafaf89b80ef9479734240aaa.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\578b6d9d88757da8481314127a4e74ea.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\579a015cad1c671bcf0b625f6dbfb65d.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\57ca4395684c2a574e2b2ff211ea86db.opus
C:\Users\VoronkovSer

C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\72042849ef4401bbc03488e5ccf76a70.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\7247e86f22c52a059dcce35eff844920.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\725d0524366a68564a68816b9eb4081d.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\72dc669ce0eb575ca68b15ada7c349d4.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\72ff15415433c6b720f4e58ee5d407a4.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\7391c0526fd460708b2842dfaa722e8e.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\73e54e925d3020cbc961f0c487032c08.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\73fe80a2950f0ae522ed2975c97b9af9.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\7444fdf02db8de4d7f3e85eb6dbc55dc.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\7459eccf0a03bfb55f04ed823d8faf40.opus
C:\Users\VoronkovSer

C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\8a18353db6f7c5a3f363c650a50935fd.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\8a9017d8b579eacd4fd9f9db4a6d8a4c.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\8a937a0d16bfc226a300c143d63257a9.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\8b2e50fec3acdc9b04ddedc5105a7de7.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\8b836cf0f2a281ce92b406f2fef0cc6d.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\8bd9c0626e015130364847dd848a1a29.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\8be382c1d66efb1a6519d697f4f7474f.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\8c37d12c69b7422599a3d1fc5df7005a.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\8cdbf69ad01fd0f771c56ae0c8aac8fc.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\8d024d126240f556a5c17db463833791.opus
C:\Users\VoronkovSer

C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\a79e51a02ccea1b450df1cbb18e54317.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\a8209151a9e960923e6c0131adf6baba.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\a83ec85dc7147f1a825b591ff148fc43.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\a87d514c3e7abc0a8360fb08a96d6f92.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\a8efab0e6dcbb753e363c91a42f6da9d.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\a9865bddb1e88445827a01a0de7bcdbd.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\a9b82df2af2902963d6acf99bff8ec57.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\aa2bc0436351393cff67705a9ff8dc48.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\aa739e8f6b2cdec1d6aae965b2ec0fcb.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\aacfa0e266e639e8c52e10a1fbe6934a.opus
C:\Users\VoronkovSer

C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\c3eddd7b02c919372fd1f4d07ad1b3af.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\c46d1357afe40081bdcd32f005577ead.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\c4bd164724c99470a6fff42c5482966c.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\c4f43aa8b79ec6c9b1fc0e86cf9c5e46.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\c55f6a93ac4b86a798a18a263cd25ed6.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\c583ef5523db9b0683407a72e3158869.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\c5ade53c1847c58e3da2625fd6281541.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\c5e1a2d408bd4d879bd29d72ae20e1f7.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\c61eeefa4def2b95de115882e60455a4.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\c6483aafbcc9efca38e9f7343ec5cf9b.opus
C:\Users\VoronkovSer

C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\de368d106dde8e3150e8c1d1f7fbc333.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\de6e1151d58a6acf52f653b9e294900e.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\dea0d5a9e1216ae7db445964f836221c.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\deed49cffad6eb439dbf2b626627f9c1.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\df515b60df4d06f52a77222bb522fc7a.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\df8fe9bac5d1196c6f7c2a206e098800.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\dfa3f55c49fe33a751066dab140b7bf8.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\dfd308021c7590e07a1343dfe198b060.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\e0635f1c168ad278457df383ee8ef07f.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\e0944f0c844c75119dc5273a0c982118.opus
C:\Users\VoronkovSer

C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\f63fe22d3ce05a9006c7637cc4479e98.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\f67278ee4196f358e53408bea7767822.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\f68d3b511d31f8549deeeb926334e323.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\f6a81bb140e1d2f3e504dc8913be80a1.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\f75fd02acf56f403a9503c25668c83b8.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\f7848771863fce198ed2bc8dfc68d430.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\f794477dd517ec747519f3ce8fe70371.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\f7f09435b671225bc974cde104b5f977.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\f8363e717ac265120fdd075d5ba0b6f3.opus
C:\Users\VoronkovSergey\IT\voice recognititon\result_data\2\f84fa7bf5d1b558266971fa981af8288.opus
C:\Users\VoronkovSer

In [44]:
path = "result_data\\2"
count = 0
for root, dirs, files in os.walk(path):
    for file in files:
        if file.split('.')[-1] == 'wav':
            count += 1
#             print(file)
count

223

In [23]:
# op = "result_data\test.opus"

opus_path = r"C:\Users\VoronkovSergey\IT\voice recognititon\result_data\test.opus"
wav_path = r"C:\Users\VoronkovSergey\IT\voice recognititon\result_data\test_2.wav"
os.system(f'ffmpeg -i "{opus_path}" -vn "{wav_path}"')


1

In [8]:
from pydub import AudioSegment

In [19]:
os.path.exists(r"result_data\test.opus")

True

In [22]:
given_audio = AudioSegment.from_file(r"result_data\test22.wav", format="wav")
given_audio.export(r"result_data\\output_audio.mp3", format="mp3")

FileNotFoundError: [WinError 2] Не удается найти указанный файл